# 1. Data Wrangling 

In [3]:
import pandas as pd
import numpy as np

In [5]:
watch_history = pd.read_csv("watch_history.csv")
users = pd.read_csv("users.csv")
search_logs = pd.read_csv("search_logs.csv")
reviews = pd.read_csv("reviews.csv")
movies = pd.read_csv("movies.csv")
recommendation_logs = pd.read_csv("recommendation_logs.csv")

In [13]:
# Create dictionary to allow looping over all datasets at once
datasets = {
    "watch_history": watch_history,
    "users": users,
    "search_logs": search_logs,
    "reviews": reviews,
    "movies": movies,
    "recommendation_logs": recommendation_logs
    }

In [15]:
# Inspect datasets
for name, df in datasets.items():
    print(f"\n=== {name.upper()} ===")
    print("Shape:", df.shape)
    print(df.head(3))
    print(df.info())
    print("Missing values:\n", df.isna().sum())
    print("-"*50)


=== WATCH_HISTORY ===
Shape: (105000, 12)
       session_id     user_id    movie_id  watch_date device_type  \
0  session_000001  user_07271  movie_0511  2025-11-13      Tablet   
1  session_000002  user_00861  movie_0588  2025-02-26      Laptop   
2  session_000003  user_05391  movie_0694  2024-12-15     Desktop   

   watch_duration_minutes  progress_percentage     action quality  \
0                    63.9                 34.6  completed      HD   
1                   120.1                 44.2    started      HD   
2                   572.1                 84.7    started      HD   

  location_country  is_download  user_rating  
0              USA        False          NaN  
1              USA        False          NaN  
2           Canada        False          1.0  
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 105000 entries, 0 to 104999
Data columns (total 12 columns):
 #   Column                  Non-Null Count   Dtype  
---  ------                  --------------   ----

In [25]:
# Drop duplicates by unique IDs
users = users.drop_duplicates(subset="user_id", keep="first")
movies = movies.drop_duplicates(subset="movie_id", keep="first")
watch_history = watch_history.drop_duplicates(subset="session_id", keep="first")
search_logs = search_logs.drop_duplicates(subset="search_id", keep="first")
reviews = reviews.drop_duplicates(subset="review_id", keep="first")
recommendation_logs = recommendation_logs.drop_duplicates(subset="recommendation_id", keep="first")

In [29]:
# Print results
print("Duplicates remaining:")
print("watch_history:", watch_history.duplicated().sum())
print("users:", users.duplicated().sum())
print("search_logs:", search_logs.duplicated().sum())
print("reviews:", reviews.duplicated().sum())
print("movies:", movies.duplicated().sum())
print("recommendation_logs:", recommendation_logs.duplicated().sum())

Duplicates remaining:
watch_history: 0
users: 0
search_logs: 0
reviews: 0
movies: 0
recommendation_logs: 0


In [65]:
# Handle missing values
# Create indicator flags before imputing (keeps info about original missingness)

# Watch history
watch_history["had_user_rating"] = np.where(watch_history["user_rating"].notna(), 1, 0)

# Use assignment, not inplace=True, for compatibility with pandas 3.0
watch_history["watch_duration_minutes"] = watch_history["watch_duration_minutes"].fillna(0)
watch_history["progress_percentage"]     = watch_history["progress_percentage"].fillna(0)
watch_history["user_rating"]             = watch_history["user_rating"].fillna(0)  # 0 = no rating recorded

# Users
users["age"]             = users["age"].fillna(users["age"].median())
users["gender"]          = users["gender"].fillna("Unknown")
users["monthly_spend"]   = users["monthly_spend"].fillna(users["monthly_spend"].median())
users["household_size"]  = users["household_size"].fillna(users["household_size"].median())

# Search logs
search_logs["clicked_result_position"] = search_logs["clicked_result_position"].fillna(0)
search_logs["clicked"]                 = np.where(search_logs["clicked_result_position"] > 0, 1, 0)
search_logs["search_duration_seconds"] = search_logs["search_duration_seconds"].fillna(
    search_logs["search_duration_seconds"].median()
)

# Reviews
reviews["helpful_votes"]  = reviews["helpful_votes"].fillna(0)
reviews["total_votes"]     = reviews["total_votes"].fillna(0)
reviews["review_text"]     = reviews["review_text"].fillna("")

# Replace sentiment score with if any non-nulls exist, else 0 
reviews["sentiment_score"] = (
    reviews["sentiment_score"].fillna(reviews["sentiment_score"].median())
    if reviews["sentiment_score"].notna().any()
    else reviews["sentiment_score"].fillna(0)
)

# Movies
movies["genre_secondary"] = movies["genre_secondary"].fillna("Unknown")

# IMDb rating: median if available, else 0
movies["imdb_rating"] = (
    movies["imdb_rating"].fillna(movies["imdb_rating"].median())
    if movies["imdb_rating"].notna().any()
    else movies["imdb_rating"].fillna(0)
)

# Fill missing numeric values with 0 and leave text columns unchanged
_movies_num_cols = movies.select_dtypes(include=["float64", "int64"]).columns
movies[_movies_num_cols] = movies[_movies_num_cols].apply(lambda s: s.fillna(0))

# Recommendation logs
recommendation_logs["recommendation_score"] = (
    recommendation_logs["recommendation_score"].fillna(recommendation_logs["recommendation_score"].median())
    if recommendation_logs["recommendation_score"].notna().any()
    else recommendation_logs["recommendation_score"].fillna(0)
)
recommendation_logs["algorithm_version"] = recommendation_logs["algorithm_version"].fillna("Unknown")

# Verify
for _name, _df in {
    "watch_history": watch_history,
    "users": users,
    "search_logs": search_logs,
    "reviews": reviews,
    "movies": movies,
    "recommendation_logs": recommendation_logs
}.items():
    remaining = int(_df.isna().sum().sum())
    print(f"{_name}: remaining NaNs = {remaining}")
    if remaining:
        print(_df.isna().sum()[_df.isna().sum() > 0].sort_values(ascending=False), "\n")

watch_history: remaining NaNs = 0
users: remaining NaNs = 0
search_logs: remaining NaNs = 0
reviews: remaining NaNs = 0
movies: remaining NaNs = 0
recommendation_logs: remaining NaNs = 0


In [45]:
# Create churn label
users["churn"] = users["is_active"].apply(lambda x: 0 if x else 1)

In [67]:
# Convert subscription_start_date from string to datetime
users["subscription_start_date"] = pd.to_datetime(users["subscription_start_date"])

# Calculate tenure (in days) up to a fixed reference date
users["tenure_days"] = (pd.to_datetime("2025-01-01") - users["subscription_start_date"]).dt.days

In [69]:
# Aggregate per user

# From Watch history
watch_features = (
    watch_history.groupby("user_id")
    .agg(
        total_watch_time=("watch_duration_minutes", "sum"),
        avg_watch_time=("watch_duration_minutes", "mean"),
        completion_rate=("progress_percentage", "mean"),
        num_sessions=("session_id", "count"),
        rated_movies=("has_rating", "sum")
    )
    .reset_index()
)

# From Search logs
search_features = (
    search_logs.groupby("user_id")
    .agg(
        search_count=("search_id", "count"),
        avg_results_returned=("results_returned", "mean"),
        click_rate=("clicked", "mean"),
        avg_search_duration=("search_duration_seconds", "mean")
    )
    .reset_index()
)

# From Reviews
review_features = (
    reviews.groupby("user_id")
    .agg(
        avg_rating=("rating", "mean"),
        review_count=("review_id", "count"),
        avg_sentiment=("sentiment_score", "mean")
    )
    .reset_index()
)

# From Recommendation logs
rec_features = (
    recommendation_logs.groupby("user_id")
    .agg(
        rec_count=("recommendation_id", "count"),
        rec_clicks=("was_clicked", "sum"),
        rec_ctr=("was_clicked", "mean")
    )
    .reset_index()
)

In [73]:
# Merge all features
user_features = users.copy()

for feats in [watch_features, search_features, review_features, rec_features]:
    user_features = user_features.merge(feats, on="user_id", how="left")

# Fill any remaining NaNs with 0 (accounts for users with no searches, reviews or recommendations)
user_features = user_features.fillna(0)

print(user_features.head())

      user_id                      email first_name last_name   age  gender  \
0  user_00001   figueroajohn@example.org      Erica     Garza  43.0    Male   
1  user_00002      blakeerik@example.com     Joshua   Bernard  38.0    Male   
2  user_00003        smiller@example.net    Barbara  Williams  32.0  Female   
3  user_00004  mitchellclark@example.com    Chelsea  Ferguson  11.0    Male   
4  user_00005      richard13@example.net      Jason    Foster  21.0  Female   

  country state_province                city subscription_plan  ...  \
0     USA  Massachusetts  North Jefferyhaven             Basic  ...   
1     USA          Texas      North Noahstad          Premium+  ...   
2     USA       Michigan          Traciebury          Standard  ...   
3     USA           Ohio          South Noah          Standard  ...   
4     USA        Arizona         West Donald          Standard  ...   

  search_count  avg_results_returned  click_rate avg_search_duration  \
0          1.0            

In [75]:
# Create and save master dataset
user_features.to_csv("user_features.csv", index=False)

## Data wrangling summary

In this step, I prepared the Netflix dataset for analysis and modeling.  

**Data loading**  
   - Imported all six raw datasets: 'users', watch_history', 'search_logs', 'reviews', 'movies', and recommendation_logs'.

**Data inspection**  
   - Checked shape, data types and missing values for each dataset.  
   - Identified duplicates and null values.
     
**Duplicates**  
   - Removed duplicates based on unique identifiers ('user_id', 'session_id', 'search_id', 'review_id', 'movie_id', 'recommendation_id') to avoid overcounting.

**Missing values**  
   - Imputed continuous variables with median values: 'age', 'monthly_spend'... 
   - Filled sparse categorical fields with "Unknown".  
   - Replaced missing votes, ratings and durations with 0.
   - Added indicator flags ('had_user_rating', 'clicked') to preserve information about whether data was originally missing.

**Feature engineering**  
   - Created churn label ('is_active=False' means 'churn=1').  
   - Computed tenure in days since subscription start.  
   - Aggregated user-level engagement features:
     - From watch history: total/average watch time, completion rate, number of sessions, number of rated movies.  
     - From search logs: search count, click rate, average search duration.  
     - From reviews: average rating given, review count, average sentiment score.  
     - From recommendation logs: number of recommendations, clicks, click-through rate.

**Creation of a master dataset**  
   - Merged all engineered features with the 'users' table into a single user-level dataset ('user_features').  
   - Verified that no missing values remain.  
   - Exported 'user_features.csv' for future analysis.

### Outcome

This dataset is now clean and ready for Exploratory Data Analysis (EDA), where I will compare active vs churned users, explore features and identify behavioral drivers of churn.